In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pybear/cats-vs-dogs")

print("Path to dataset files:", path)

100%|██████████| 788M/788M [00:05<00:00, 147MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/pybear/cats-vs-dogs/versions/1


In [2]:
import os
import shutil
from PIL import Image


In [3]:
raw_images = path + "/PetImages/"
cleaned_images = path + "/PetImages/cleaned"

os.makedirs(cleaned_images, exist_ok=True)


In [4]:
for categories in ['Cat', 'Dog']:
    os.makedirs(os.path.join(cleaned_images , categories), exist_ok= True)
    source_folder = os.path.join(raw_images , categories)
    cleaned_folder = os.path.join(cleaned_images , categories)


    for img_name in os.listdir(source_folder):
        source_path = os.path.join(source_folder, img_name)
        cleaned_path = os.path.join(cleaned_folder , img_name)
        try:
            with Image.open(source_path) as img:
                img = img.convert("RGB")
                img.save(cleaned_path)
        except Exception as e:
            print(f"corrupted file cant be open: {source_path}")



corrupted file cant be open due to exception /root/.cache/kagglehub/datasets/pybear/cats-vs-dogs/versions/1/PetImages/Cat/Thumbs.db
corrupted file cant be open due to exception /root/.cache/kagglehub/datasets/pybear/cats-vs-dogs/versions/1/PetImages/Cat/666.jpg
corrupted file cant be open due to exception /root/.cache/kagglehub/datasets/pybear/cats-vs-dogs/versions/1/PetImages/Dog/11702.jpg
corrupted file cant be open due to exception /root/.cache/kagglehub/datasets/pybear/cats-vs-dogs/versions/1/PetImages/Dog/Thumbs.db


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [12]:
import random
import matplotlib.pyplot as plt

def view_images_cat(count = 10):
    folder_cat = f"{path}/PetImages/cleaned/Cat"
    files_name = os.listdir(folder_cat)
    random_file = random.sample(files_name, count)

    for i , file in enumerate(random_file):
        path_ = os.path.join(folder_cat, file)
        plt.figure(figsize=(3, 3))
        img = Image.open(path_)
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout()
    plt.show()


def view_images_dog(count=10):
    folder_Dog = f"{path}/PetImages/cleaned/Dog"
    files_name = os.listdir(folder_Dog)
    random_file = random.sample(files_name, count)

    for i , file in enumerate(random_file):
        path_ = os.path.join(folder_Dog , file)
        plt.figure(figsize=(3, 3))
        img = Image.open(path_)
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout()
    plt.show()


# view_images_cat()
# view_images_dog()

In [ ]:
import tensorflow as tf
